<a href="https://colab.research.google.com/github/benjaminADMU/NBO_BPI/blob/main/Recommendable_Products.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import numpy as np

### Load Dataset

In [2]:
df = pd.read_csv('../Data/Products.csv',header=0)
pq = pd.read_parquet('../Data/Product Flags/cust_info_prod_202403_processed.parquet')
cpq = pd.read_parquet('../Data/Other Data/CUSTOMER INFO.parquet')
merged_df = pd.merge(pq, cpq, on='CUST_NUM', how='left')

### Get Current Products

In [3]:
CURRENT_PRODUCTS = []

for index, row in merged_df.iterrows():
    client_number = row['CUST_NUM']
    products = row[2:18]
    current_product_list = products[products == 1].index.tolist()
    CURRENT_PRODUCTS.append((client_number, current_product_list))

In [4]:
product_columns = pq.columns[2:18]

merged_df['CURRENT_PRODUCTS'] = merged_df[product_columns].apply(
    lambda row: [col for col in product_columns if row[col] == 1], axis=1
)

merged_df = merged_df.drop(columns=product_columns)

In [5]:
eligibility_features = ['CUST_NUM','AGE', 'TENURE','INCOME_SOURCE','SUBSEGMENT','BUSINESS_OWNER', 'IDV_OR_ORG', 'CURRENT_PRODUCTS']
merged_df = merged_df[eligibility_features]

In [6]:
columns_to_append = ['AGE', 'TENURE', 'INCOME_SOURCE', 'SUBSEGMENT',
                     'BUSINESS_OWNER', 'IDV_OR_ORG', 'CURRENT_PRODUCTS',
                     'CREDIT_SCORE', 'DEBT_TO_INCOME_RATIO',
                     'EXISTING_DEBTS',
                     'DEFAULTS', 'ON_TIME_PAYMENTS']

for column in columns_to_append:
    df[column] = pd.NA

#### Synthetic Data Generator

In [7]:
class SyntheticDataGenerator:
    def __init__(self, num_samples=100):
        self.num_samples = num_samples
        self.income_levels = {
            'Lower Tier': (15000, 25000),
            'Middle Tier': (25001, 40000),
            'Upper-Middle Tier': (40001, 60000),
            'Top Tier': (60001, 100000)
        }

    def generate_income_and_levels(self):
        income_category = np.random.choice(list(self.income_levels.keys()), size=self.num_samples)
        income = []
        for category in income_category:
            lower, upper = self.income_levels[category]
            income.append(np.random.randint(lower, upper + 1))
        return np.array(income), income_category

    def generate_credit_scores(self):
        credit_scores = np.random.normal(loc=600, scale=50, size=self.num_samples)
        return np.clip(np.round(credit_scores), 300, 850).astype(int)

    def generate_existing_debts(self):
        existing_debts = np.random.lognormal(mean=np.log(20000), sigma=1, size=self.num_samples)
        return existing_debts.astype(int)

    def generate_debt_to_income_ratios(self, existing_debts, income):
        monthly_debt_payments = existing_debts / 12
        dti_ratios = monthly_debt_payments / (income / 12)
        return np.clip(dti_ratios, 0, 1)

    def generate_payment_history(self):
        total_payments = np.random.randint(5, 15, self.num_samples)
        defaults = np.random.poisson(lam=0.5, size=self.num_samples)
        return total_payments, defaults

    def generate_data(self):
        income, income_levels = self.generate_income_and_levels()
        existing_debts = self.generate_existing_debts()
        credit_scores = self.generate_credit_scores()
        dti_ratios = self.generate_debt_to_income_ratios(existing_debts, income)
        total_payments, defaults = self.generate_payment_history()

        data = {
            'INCOME_LEVEL': income_levels,
            'CREDIT_SCORE': credit_scores,
            'DEBT_TO_INCOME_RATIO': dti_ratios,
            'EXISTING_DEBTS': existing_debts,
            'TOTAL_PAYMENTS': total_payments,
            'DEFAULTS': defaults,
            'ON_TIME_PAYMENTS': total_payments - defaults,
            'INCOME': income
        }

        return pd.DataFrame(data)



#### Add Synthetic Data in the Customer Dataset

In [8]:
# Initialize the synthetic data generator
generator = SyntheticDataGenerator(num_samples=len(merged_df))

# Generate synthetic data
synthetic_data = generator.generate_data()

# Combine the existing DataFrame with the synthetic data
combined_df = pd.concat([merged_df.reset_index(drop=True), synthetic_data.reset_index(drop=True)], axis=1)

In [9]:
# Display the combined DataFrame
combined_df.head()

,CUST_NUM,AGE,TENURE,INCOME_SOURCE,SUBSEGMENT,BUSINESS_OWNER,IDV_OR_ORG,CURRENT_PRODUCTS,INCOME_LEVEL,CREDIT_SCORE,DEBT_TO_INCOME_RATIO,EXISTING_DEBTS,TOTAL_PAYMENTS,DEFAULTS,ON_TIME_PAYMENTS,INCOME
0,1357.977909,22.0,1.200000,SALARY,Lower Tier,N,INDIVIDUAL,"[BB, DEPOSITS]",Lower Tier,568,0.377955,8905,14,3,11,23561
1,1437.499217,23.0,5.879452,ALLOWANCE,Lower Tier,N,INDIVIDUAL,"[BB, DEPOSITS]",Middle Tier,582,0.278472,8778,10,3,7,31522
2,1606.444521,13.0,2.345205,ALLOWANCE,Upper-Middle Tier,N,INDIVIDUAL,"[BB, DEPOSITS]",Middle Tier,661,0.400342,11696,12,0,12,29215
3,2522.626409,18.0,3.043836,ALLOWANCE,Lower Tier,N,INDIVIDUAL,"[BB, DEPOSITS]",Upper-Middle Tier,612,1.000000,91254,7,0,7,58165
4,3201.700173,22.0,1.528767,INTEREST_SAVINGS_PLACEMENTS_INVESTMENTS,Upper-Middle Tier,N,INDIVIDUAL,"[BB, DEPOSITS]",Lower Tier,643,0.160238,3926,10,0,10,24501


### Generate Eligibility Rules

#### Rename df for convenience

In [10]:
product_df = df
customer_df = combined_df

#### Generate values for criteria to be replaced when data is available

In [11]:
# Define ranges based on reasonable assumptions for eligibility criteria
def generate_eligibility_values(product_df):
    np.random.seed(0)  # For reproducibility

    # Generate AGE eligibility
    product_df.loc[product_df['AGE'].isna(), 'AGE'] = 18

    # Generate TENURE eligibility between 1 and 4 years
    product_df.loc[product_df['TENURE'].isna(), 'TENURE'] = np.random.randint(1, 4, size=product_df['TENURE'].isna().sum())

    # Populate categorical fields based on possible values observed in customers
    income_sources = ['Salary', 'Business', 'Pension', 'Investment']
    product_df.loc[product_df['INCOME_SOURCE'].isna(), 'INCOME_SOURCE'] = np.random.choice(income_sources, product_df['INCOME_SOURCE'].isna().sum())

    subsegments = ['Lower Tier', 'Middle Tier', 'Upper-Middle Tier', 'Top Tier']
    product_df.loc[product_df['SUBSEGMENT'].isna(), 'SUBSEGMENT'] = np.random.choice(subsegments, product_df['SUBSEGMENT'].isna().sum())

    product_df.loc[product_df['BUSINESS_OWNER'].isna(), 'BUSINESS_OWNER'] = np.random.choice([True, False], product_df['BUSINESS_OWNER'].isna().sum())
    product_df.loc[product_df['IDV_OR_ORG'].isna(), 'IDV_OR_ORG'] = np.random.choice(['Individual', 'Organization'], product_df['IDV_OR_ORG'].isna().sum())

    # CREDIT_SCORE eligibility
    product_df.loc[product_df['CREDIT_SCORE'].isna(), 'CREDIT_SCORE'] = np.random.randint(300, 400, size=product_df['CREDIT_SCORE'].isna().sum())

    # DEBT_TO_INCOME_RATIO eligibility
    product_df.loc[product_df['DEBT_TO_INCOME_RATIO'].isna(), 'DEBT_TO_INCOME_RATIO'] = np.random.uniform(0, 5, size=product_df['DEBT_TO_INCOME_RATIO'].isna().sum()).round(2)

    # EXISTING_DEBTS eligibility
    product_df.loc[product_df['EXISTING_DEBTS'].isna(), 'EXISTING_DEBTS'] = np.random.uniform(0,1000 , size=product_df['EXISTING_DEBTS'].isna().sum()).round(2)

    # DEFAULTS eligibility from 0 to 5 (number of defaults)
    product_df.loc[product_df['DEFAULTS'].isna(), 'DEFAULTS'] = np.random.randint(0, 5, size=product_df['DEFAULTS'].isna().sum())

    # ON_TIME_PAYMENTS eligibility
    product_df.loc[product_df['ON_TIME_PAYMENTS'].isna(), 'ON_TIME_PAYMENTS'] = np.random.uniform(0, 5, size=product_df['ON_TIME_PAYMENTS'].isna().sum()).round(2)

    return product_df


In [12]:
product_df = generate_eligibility_values(product_df)

/var/folders/jv/0sr2mpzx5klfxvmfj4s39_vm0000gn/T/ipykernel_32532/3429502202.py:9: DeprecationWarning: In a future version, `df.iloc[:, i] = newvals` will attempt to set the values inplace instead of always setting a new array. To retain the old behavior, use either `df[df.columns[i]] = newvals` or, if columns are non-unique, `df.isetitem(i, newvals)`
  product_df.loc[product_df['TENURE'].isna(), 'TENURE'] = np.random.randint(1, 4, size=product_df['TENURE'].isna().sum())
/var/folders/jv/0sr2mpzx5klfxvmfj4s39_vm0000gn/T/ipykernel_32532/3429502202.py:18: DeprecationWarning: In a future version, `df.iloc[:, i] = newvals` will attempt to set the values inplace instead of always setting a new array. To retain the old behavior, use either `df[df.columns[i]] = newvals` or, if columns are non-unique, `df.isetitem(i, newvals)`
  product_df.loc[product_df['BUSINESS_OWNER'].isna(), 'BUSINESS_OWNER'] = np.random.choice([True, False], product_df['BUSINESS_OWNER'].isna().sum())
/var/folders/jv/0sr2mp

### Create a checker if a customer is eligible

In [13]:
def check_product_eligibility(customer_df, product_df):
    eligibility_results = []

    # Go through each product and evaluate eligibility in vectorized form
    for _, product in product_df.iterrows():
        # Define conditions with fallbacks (use fillna to handle missing values if necessary)
        age_eligible = customer_df['AGE'] >= product['AGE']
        tenure_eligible = customer_df['TENURE'] >= product['TENURE']
        credit_score_eligible = customer_df['CREDIT_SCORE'] >= product['CREDIT_SCORE']
        defaults_eligible = customer_df['DEFAULTS'] <= product['DEFAULTS']

        # Check if the product is not already owned, or if "current_products" is empty
        product_not_owned = customer_df['CURRENT_PRODUCTS'].apply(lambda x: product['Product_Name'] not in x) if 'current_products' in customer_df else True

        # Combine all criteria into a single eligibility condition
        eligible_customers = customer_df[age_eligible & tenure_eligible & credit_score_eligible & defaults_eligible & product_not_owned]

        # For eligible customers, append the product name to their eligibility list
        for cust_num in eligible_customers['CUST_NUM']:
            eligibility_results.append({'CUST_NUM': cust_num, 'Eligible_Product': product['Product_Name']})

    # Create a DataFrame from the eligibility results list
    eligibility_df = pd.DataFrame(eligibility_results)

    # Aggregate eligible products for each customer
    eligibility_summary_df = eligibility_df.groupby('CUST_NUM')['Eligible_Product'].apply(list).reset_index()
    eligibility_summary_df.columns = ['CUST_NUM', 'Eligible_Products']

    return eligibility_summary_df




In [14]:
# Usage example:
eligibility_df = check_product_eligibility(customer_df, product_df)
eligibility_df.head()

,CUST_NUM,Eligible_Products
0,8.000000,"[Corporate Credit Cards, Corporate Credit Card..."
1,36.633318,"[Corporate Credit Cards, Corporate Credit Card..."
2,37.735925,"[Corporate Credit Cards, Corporate Credit Card..."
3,47.770284,"[Corporate Credit Cards, Corporate Credit Card..."
4,53.141321,"[Corporate Credit Cards, Corporate Credit Card..."


### Create dataframe for the list of customers available for each product

In [15]:
# Initialize a dictionary to store products as keys and list of CUST_NUM as values
product_customers = {}

# Iterate over each row in eligibility_df
for _, row in eligibility_df.iterrows():
    cust_num = row['CUST_NUM']
    for product in row['Eligible_Products']:
        # Add the CUST_NUM to the product's list in the dictionary
        if product not in product_customers:
            product_customers[product] = []
        product_customers[product].append(cust_num)

# Convert the dictionary to a DataFrame where each product is a column
cproduct_df = pd.DataFrame(dict([(k, pd.Series(v)) for k, v in product_customers.items()]))



In [16]:
# Display the final DataFrame
cproduct_df.head()

,Corporate Credit Cards,Corporate Credit Cards - Credit,Corporate Credit Cards - Prepaid,Housing loans,Other loans,Leasing,Short-term loans,Supplier Finance,Distributor Fiannce,Investments (AMTC/BIMI),...,Auto loans,Operating lease,RPNL/STL/KNCL,Bills purchased,Duties payment,Disbursements - government,Disbursements - digital,Liquidity,Other current,Peso Time Deposit
0,8.000000,8.000000,8.000000,8.000000,8.000000,8.000000,8.000000,8.000000,8.000000,8.000000,...,47.770284,47.770284,47.770284,47.770284,47.770284,47.770284,47.770284,47.770284,47.770284,47.770284
1,36.633318,36.633318,36.633318,36.633318,36.633318,36.633318,36.633318,36.633318,36.633318,36.633318,...,53.141321,53.141321,53.141321,53.141321,53.141321,53.141321,53.141321,53.141321,53.141321,53.141321
2,37.735925,37.735925,37.735925,37.735925,37.735925,37.735925,37.735925,37.735925,37.735925,37.735925,...,56.035703,56.035703,56.035703,56.035703,56.035703,56.035703,56.035703,56.035703,56.035703,56.035703
3,47.770284,47.770284,47.770284,47.770284,47.770284,47.770284,47.770284,47.770284,47.770284,47.770284,...,65.893854,65.893854,65.893854,65.893854,65.893854,65.893854,65.893854,65.893854,65.893854,65.893854
4,53.141321,53.141321,53.141321,53.141321,53.141321,53.141321,53.141321,53.141321,53.141321,53.141321,...,80.386566,80.386566,80.386566,80.386566,80.386566,80.386566,80.386566,80.386566,80.386566,80.386566
